In [16]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from config import *
import fitz
from tqdm import tqdm
import shutil

In [8]:
import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_embedding(text, model="text-embedding-3-small"):
    response = client.embeddings.create(input=[text], model=model)
    return np.array(response.data[0].embedding)

def cos_sim(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]

# Define the test texts
inclusion_text = "studies using simulation modeling such as agent-based models, Monte Carlo, and Markov chains."
exclusion_text = "studies using only regression analysis or descriptive statistics without simulation."
combined_text = inclusion_text + " \n " + exclusion_text
chunk_text = "This study uses logistic regression to model heart disease outcomes and not just regression analysis."

# Get embeddings
embedding_inclusion = get_embedding(inclusion_text)
embedding_exclusion = get_embedding(exclusion_text)
embedding_combined  = get_embedding(combined_text)
embedding_chunk     = get_embedding(chunk_text)

# Compute similarities
sim_incl = cos_sim(embedding_chunk, embedding_inclusion)
sim_excl = cos_sim(embedding_chunk, embedding_exclusion)
sim_comb = cos_sim(embedding_chunk, embedding_combined)

# Print results
print("Cosine Similarity (Chunk ↔ Inclusion):", round(sim_incl, 4))
print("Cosine Similarity (Chunk ↔ Exclusion):", round(sim_excl, 4))
print("Cosine Similarity (Chunk ↔ Combined) :", round(sim_comb, 4))
print("Contrastive Relevance Score (incl - excl):", round(sim_incl - sim_excl, 4))


Cosine Similarity (Chunk ↔ Inclusion): 0.319
Cosine Similarity (Chunk ↔ Exclusion): 0.495
Cosine Similarity (Chunk ↔ Combined) : 0.4291
Contrastive Relevance Score (incl - excl): -0.176


In [77]:
OUTPUT_FOLDER = "../data/output"
output_files = os.listdir(OUTPUT_FOLDER)

In [78]:
compliant_files = []
compliant_dict = {}

for filename in tqdm(output_files):
    if not filename.endswith("_highlighted.pdf"):
        continue
        
    pdf_path = os.path.join(OUTPUT_FOLDER, filename)
    
    # Read the PDF and check for highlights
    doc = fitz.open(pdf_path)
    found_colors = set()
    has_highlights = False
    
    # Initialize criteria flags for this file
    criteria_met = {
        'Population': False,
        'Intervention': False,
        'Outcome': False,
        'Study approach': False
    }
    
    # Check each page for highlights
    for page in doc:
        highlights = page.annots()
        
        for highlight in highlights:
            if highlight.type[0] != fitz.PDF_ANNOT_HIGHLIGHT:
                continue
                
            has_highlights = True
            color = highlight.colors["stroke"]
            # Convert color tuple to match CRITERIA_COLORS format
            color_tuple = (round(color[0], 1), round(color[1], 1), round(color[2], 1))
            
            # Check if highlight has a comment starting with "YES"
            comment = highlight.info.get("content", "")
            if comment.startswith("YES"):
                # Add color to set if it's one of the first 4 criteria colors
                if color_tuple in list(CRITERIA_COLORS.values())[:4]:
                    found_colors.add(color_tuple)
                    # Track which criteria are met
                    if color_tuple == list(CRITERIA_COLORS.values())[0]:  # Population
                        criteria_met['Population'] = True
                    elif color_tuple == list(CRITERIA_COLORS.values())[1]:  # Intervention
                        criteria_met['Intervention'] = True
                    elif color_tuple == list(CRITERIA_COLORS.values())[2]:  # Outcome
                        criteria_met['Outcome'] = True
                    elif color_tuple == list(CRITERIA_COLORS.values())[3]:  # Study approach
                        criteria_met['Study approach'] = True

    # Store criteria for all files with highlights
    if has_highlights:
        compliant_dict[filename] = criteria_met
        
        # Check if all 4 colors were found for compliant files list
        if len(found_colors) == 4:
            compliant_files.append(filename)
        
    doc.close()

# Create DataFrame showing all files and their criteria status
df_compliant = pd.DataFrame.from_dict(compliant_dict, orient='index')

# Add a column showing if all criteria are met
df_compliant['All Criteria Met'] = df_compliant.all(axis=1)
df_compliant = df_compliant.reset_index(names='File')

# Display summary statistics
print(f"Found {len(compliant_files)} compliant files:")
for file in compliant_files:
    print(f"- {file}")

print(f"\nTotal files analyzed: {len(df_compliant)}")

for criterion in df_compliant.columns[1:-1]:
    count = df_compliant[criterion].sum()
    print(f"{criterion}: {count} files")

print(f"\nFiles meeting all criteria: {df_compliant['All Criteria Met'].sum()}")


100%|██████████| 211/211 [00:50<00:00,  4.19it/s]

Found 13 compliant files:
- Archbold et al. - 2024 - An agent-based model of the spread of behavioural _highlighted.pdf
- Basu, Wagner, Sewpaul, Reddy, Davies_The Lancet Global Health_2019_highlighted.pdf
- Burden-of-disease-scenarios-for-204-countries-and-_highlighted.pdf
- Burke, Copeland, Sussman, Hayward, Gross, Briceno, Whitney, Giordani, Elkind, Manly, Gottesman, Gaskin, Sidney, Yaffe, Sacco, Heckbert, Hughes, Galecki, Levine_PLOS One_2024_highlighted.pdf
- Global-burden-and-strength-of-evidence-for-88-risk_highlighted.pdf
- Head et al. - The IMPACTNCD technical appendix_highlighted.pdf
- Iorfino, Occhipinti, Skinner, Davenport, Rowe, Prodan, Sturgess, Hickie_J. Med. Internet Res._2021_highlighted.pdf
- Koutouan - 2023 - The Effect of Screening Behavior Assumptions and H_highlighted.pdf
- Marklund, Aminde, Wanjau, Ale, Ojo, Okoro, Adegboye, Huang, Veerman, Wu, Huffman, Ojji_BMJ Glob. Health_2024_highlighted.pdf
- Mihaylova, Wu, Zhou, Williams, Schlackow, Emberson, Reith, Keech, R

In [79]:
df_compliant

,File,Population,Intervention,Outcome,Study approach,All Criteria Met
0,"Aballea, Beck, Cheng, Demarteau, Li, Ma, Neine...",False,False,True,True,False
1,"Ali, Li, Moqueet, Moghadas, Galvani, Cooper, S...",False,False,False,False,False
2,"Alimohammadi, Chaovalitwongse, Vesselle, Zhang...",False,False,False,False,False
3,Allen et al. - 2020 - Bayesian hierarchical do...,True,True,False,True,False
4,"Alqahtani, Al-Omar, Alshehri, Abanumay, Alabdu...",True,False,True,False,False
...,...,...,...,...,...,...
178,"Zhang, Zafari, Slejko, Castillo, Reeves, dosRe...",True,False,False,True,False
179,Zheng et al. - 2023 - The clinicopathological ...,False,False,False,False,False
180,"Zhou, Shen_Lancet_2024_highlighted.pdf",False,True,True,False,False
181,Zhu et al. - 2023 - Treatment-related adverse ...,False,False,True,False,False


In [80]:
df_compliant.to_excel("../data/excels/compliant_files.xlsx", index=False)

In [11]:
df = pd.read_csv("../data/excels/Exported Items.csv")
df.head(1)

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,...,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
0,7Z3S4GYD,journalArticle,2021.0,"Jayasekera, Jinani; Sparano, Joseph A.; O'Neil...",Development and Validation of a Simulation Mod...,Journal of Clinical Oncology,NaN,0732-183X,10.1200/JCO.21.00651,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_human_included = df[df['Manual Tags'].str.contains('obj2_include')][['Manual Tags', 'File Attachments', 'Title']]
df_human_included.shape

(75, 3)

In [13]:
df_human_included.iloc[5]['File Attachments']

'C:\\Users\\pouri\\Zotero\\storage\\WQWJYC62\\Emmert-Fees et al. - 2023 - Projected health and economic impacts of sugar-swe.pdf'

In [45]:
df_human_included[df_human_included['File Attachments'].str.contains('The modelled impact of')]

,Manual Tags,File Attachments,Title
194,Humans; Female; Adult; Male; Quality-Adjusted ...,; C:\Users\pouri\Zotero\storage\2DNGECPZ\Mihay...,Assessing long-term effectiveness and cost-eff...


In [29]:
import os
import shutil

# Paths
archive_path = r'C:\Users\pouri\Python\Lang_Chain\Projects\NLP_pipeline_full_text\data\papers\Archive'
destination_path = r'C:\Users\pouri\Python\Lang_Chain\Projects\NLP_pipeline_full_text\data\papers'

# Ensure destination path exists
os.makedirs(destination_path, exist_ok=True)

# Normalize filenames from the DataFrame column to a set
included_files = set(df_human_included['File Attachments'].astype(str).str.split('\\').str[-1].str.strip())

# Loop through files and copy if in included list
for file in os.listdir(archive_path):
    if file.strip() in included_files:
        shutil.copy2(os.path.join(archive_path, file), os.path.join(destination_path, file))
